In [1]:
from amazon.backend.database import *
from amazon.backend import socketUtils, UPSMessage, WorldMessage, request, query
from amazon.backend import amazon_ups_pb2 as UPS
from amazon.backend import world_amazon_pb2 as WORLD
from PIL import Image
import io
import copy

import struct

In [2]:
package_id = 0
def buy_product(user_id, product_id, amount, address):
    sock = socketUtils.socket_connect("vcm-30469.vm.duke.edu",29081)
    # modify databse，generate packageid,
    global package_id
    engine = getEngine()
    Session = sessionmaker(bind=engine)
    session = Session()

    # need lock
    # package_id =session.execute(select(func.max(Order.id))).scalar()+1
    package_id += 1
    # 
    neworder = Order(buyer = user_id, product_id = product_id, amount = amount, status = 'packing', package = package_id)
    session.add(neworder)
    session.commit()
    product = session.query(Products).filter(Products.id == product_id).first()

    value = struct.pack('!I', package_id)
    sock.sendall(value)
    value = struct.pack('!I', address[0])
    sock.sendall(value)
    value = struct.pack('!I', address[1])
    sock.sendall(value)
    
    d = sock.recv(4)
    length = struct.unpack('!I', d)[0]
    message = sock.recv(length).decode()
    print(message)
    if(message != 'Success'):
        return message
    

    print(user_id,product_id,amount,address) 
    pass

In [3]:
buy_product(1, 0, 10, (1,2))

connect to vcm-30469.vm.duke.edu


error: unpack requires a buffer of 4 bytes